# CSC421 Assignment 4 - Part I Discrete Bayesian Networks (5 points) #
### Author: George Tzanetakis 

This notebook is based on the supporting material for topics covered in **Chapter 14 Probabilistic Reasoning** from the book *Artificial Intelligence: A Modern Approach.* 

This part relies on the provided notebook code probability.ipynb 

```
Misunderstanding of probability may be the greatest of all impediments
to scientific literacy.

Gould, Stephen Jay
```



## Introduction 

In this part of assignment 4 we will be exploring Discrete Bayesian Networks (DBNs) and answering queries with exact and approximate inference methods. We will be using the following network: 

<img src="dispnea.png">

## Question 4.1A (Minimum) CSC421 -  (1 point, CSC581C - 0 points) 

Using the convetions for DBNs used in probability.ipynb (from the AIMA authors) encode the diapnea network shown above. Once you have constructed the Bayesian network display the cpt for the Lung Cancer Node (using the API provided not just showing the numbers). 


In [123]:
from probability import *
from utils import print_table
from tabulate import*

A = ('A', '', 0.01)
S = ('S', '', 0.5)
T = ('T', ['A'], {True: 0.05, False: 0.01})
L = ('L', ['S'], {True: 0.1, False: 0.01})
B = ('B', ['S'], {True: 0.6, False: 0.3})
E = ('E', ['L', 'T'], {(True, True): 1.0, (True, False): 1.0, (False, True): 1.0, (False, False): 0.0})
X = ('X', ['E'], {True: 0.98, False: 0.05})
D = ('D', ['E', 'B'], {(True, True): 0.9, (True, False): 0.7, (False, True): 0.8, (False, False): 0.1})

dyspnea = BayesNet([A, S, T, L, B, E, X, D])

Headers = [dyspnea.variable_node('L').parents[0], 'Probability']
Data = [['True', dyspnea.variable_node('L').cpt[(True,)]],
        ['False', dyspnea.variable_node('L').cpt[(False,)]]
       ]

print('Lung Cancer?')
print(tabulate(Data,Headers,tablefmt="fancy_grid", numalign = "right"))




Lung Cancer?
╒═══════╤═══════════════╕
│ S     │   Probability │
╞═══════╪═══════════════╡
│ True  │           0.1 │
├───────┼───────────────┤
│ False │          0.01 │
╘═══════╧═══════════════╛


## Question 4.1B (Minimum) (CSC421 - 1 point, CSC581C - 0 point) 

Answer using exact inference with enumeration the following query: given that a
patient has been in Asia and has a positive xray, what is the likelihood of having dyspnea?

Write down using markdown the expression that corresponds to this query and the corresponding 
numbers from the CPT. Calculate the result using a calculator. 

Write code for the same query using *enumeration_ask* and confirm that the result is the same for the same query. 

P(D|A,X)
= P(A)P(S)Σ(T)P(T|A)Σ(L)P(L|S)Σ(B)P(B|S)Σ(E)P(E|L,T)P(X|E)P(D|E,B)

P(T) = P(T|A)P(A) = (0.05)(1) = 0.05
P(L) = Σs P(L|S)P(S) = (0.1)(0.5) + (0.01)(0.5) = 0.055
P(B) = Σs P(B|S)P(S) = (0.6)(0.5) + (0.3)(0.5) = 0.45
P(E) = Σl,t P(E|L,T)P(L)P(T) = (1)(0.055)(0.05) + (1)(0.055)(0.95) + (1)(0.945)(0.05)
     = 0.10225
     
P(D) = Σe,b P(D|E,B)P(E)P(B)P(X|E) = (0.9)(0.10225)(0.45)(0.98) + (0.7)(0.10225)(0.55)(0.98) + (0.8)(0.89775)(0.45)(0.05) + (0.1)(0.89775)(0.55)(0.05)
     = 0.097790262

P(~D) = Σe,b P(~D|E,B)P(E)P(B)P(X|E) = (0.1)(0.10225)(0.45)(0.98) + (0.3)(0.10225)(0.55)(0.98) + (0.2)(0.89775)(0.45)(0.05) + (0.9)(0.89775)(0.55)(0.05)
      = 0.047302237
      
alpha is 0.097790262 + 0.047302237 = 0.145092499

Normalizing P(D): 0.097790262/0.145092499 = 0.673985646

P(D|A,X) = 67.4%

In [128]:
print(enumeration_ask('D', {'X': True, 'A': True}, dyspnea).show_approx())

False: 0.319, True: 0.681


## Question 4.1C (Expected) 1 point 

Answer using variable elimination i.e the function *elimination_ask*  using the same query. Compare the timing using %%timeit the query using *enumeration_ask* and *eliimination_ask*. 

In [163]:
print(elimination_ask('D', {'X': True, 'A': True}, dyspnea).show_approx(),'\n')

print('enumeration_ask runtime:')
%timeit enumeration_ask('D', {'X': True, 'A': True}, dyspnea)
print('\nelimination_ask runtime:')
%timeit elimination_ask('D', {'X': True, 'A': True}, dyspnea)


False: 0.319, True: 0.681 

enumeration_ask runtime:
6.35 ms ± 1.01 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)

elimination_ask runtime:
13.4 ms ± 250 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


## QUESTION 4.1D (Expected ) 1 point

Answer using approximate inference the same query using both rejection sampling and likelihood weighting. Compare the runtime of the two approximate inference algorithms and the two exact inference algorithms for this query. 

In [165]:
print('Rejection Sampling\n','-'*25,sep='')
print(rejection_sampling('D', {'X': True, 'A': True}, dyspnea, 10000).show_approx())
%timeit rejection_sampling('D', {'X': True, 'A': True}, dyspnea, 10000).show_approx()

print('\nLikelihood Weighting\n','-'*25,sep='')
print(likelihood_weighting('D', {'X': True, 'A': True}, dyspnea, 10000).show_approx())
%timeit likelihood_weighting('D', {'X': True, 'A': True}, dyspnea).show_approx()

Rejection Sampling
-------------------------
False: 0.4, True: 0.6
336 ms ± 7.74 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

Likelihood Weighting
-------------------------
False: 0.309, True: 0.691
309 ms ± 30.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## QUESTION 4.1E (Advanced) 1 point 

A Naive Bayes classifier can be considered as a Bayesian Network. The classification problem can then be expressed as setting all the variables corresponding to the features as evidence and querying the probability for the class. Express the Bernoulli Naive Bayes classifier you implemented in the previous assignment as a Bayesian Network using the probability.ipynb conventions. Now that you have a DBN express and solve the classification problem as a query and go over all the previous steps for this particular problem. More specifically do exact inference by enumeration, exact inference by variable elimination, approximate inference by rejection sampling and approximate inference by likelihood weighting to answer the query and show the results. 

In [214]:
pos_probs = [0.019, 0.254, 0.048, 0.023, 0.120, 0.094, 0.405, 0.125]
neg_probs = [0.099, 0.503, 0.166, 0.090, 0.046, 0.053, 0.282, 0.048]

pos_cpt = {}
neg_cpt = {}

cpt_keys = [[]]

#create a list of all possible variable assignments
for i in range(0,8):
    for item in cpt_keys[::1]:
        copy = item.copy()
        item.append(True)
        copy.append(False)
        cpt_keys.append(copy)

#complete cpt for the positive node
for cp in cpt_keys:
    prob = 1
    for i,value in enumerate(cp):
        if value == True: prob = prob*pos_probs[i]
        else: prob = prob*(1-pos_probs[i])
    pos_cpt[tuple(cp)]=prob

#complete cpt for the negative node
for cp in cpt_keys:
    prob = 1
    for i,value in enumerate(cp):
        if value == True: prob = prob*neg_probs[i]
        else: prob = prob*(1-neg_probs[i])
    neg_cpt[tuple(cp)]=prob

#word probabilities are irrelevant; we will have their values as evidence in all queries
Aw = ('awful', '', 0.059)
Ba = ('bad', '', 0.3785)
Bo = ('boring', '', 0.107)
Du = ('dull', '', 0.0565)
Ef = ('effective', '', 0.083)
En = ('enjoyable', '', 0.0735)
Gr = ('great', '', 0.3435)
Hi = ('hilarious', '', 0.0865)
Po = ('Pos', ['awful','bad','boring','dull','effective','enjoyable','great','hilarious'], pos_cpt)
Ne = ('Neg', ['awful','bad','boring','dull','effective','enjoyable','great','hilarious'], neg_cpt)

video_review = BayesNet([Aw, Ba, Bo, Du, Ef, En, Gr, Hi, Po, Ne, ])
review_sample = {'Aw': False, 
                  'Ba': False, 
                  'Bo': False,
                  'Du': False,
                  'Ef': False,
                  'En': True,
                  'Gr': True,
                  'Hi': False}

def classify(sample, method):
    print('Classification using', method.__name__)
    print('-'*30)
    pos = method('Pos', sample, video_review).show_approx()
    neg = method('Neg', sample, video_review).show_approx()
    print('Positive?',pos)
    print('Negative?',neg)
    if pos[True] >= neg[True]:
        print('Prediction: the review is positive')
    else:
        print('Prediction: the review is negative')
    print()

classify(review_sample, enumeration_ask)
classify(review_sample, elimination_ask)
classify(review_sample, rejection_sampling)
classify(review_sample, likelihood_weighting)

Classification using enumeration_ask
------------------------------
Positive? False: 0.879, True: 0.121
Negative? False: 0.892, True: 0.108
Prediction: the review is positive

Classification using elimination_ask
------------------------------
Positive? False: 0.879, True: 0.121
Negative? False: 0.892, True: 0.108
Prediction: the review is positive

Classification using rejection_sampling
------------------------------
Positive? False: 0.882, True: 0.118
Negative? False: 0.891, True: 0.109
Prediction: the review is positive

Classification using likelihood_weighting
------------------------------
Positive? False: 0.878, True: 0.122
Negative? False: 0.892, True: 0.108
Prediction: the review is positive

